## Исследование надежности заемщиков _(часть 2)_

### 3. Откроем чистые данные

In [2]:
import pandas as pd
data_cleaned = pd.read_csv('/Users/yuliabezginova/PycharmProjects/project-1_bank-credit-scoring/data_cleaned.csv')
display(data_cleaned.head(10))

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## 4. Исследуем чистые данные и ответим на исследовательские вопросы

### Вопрос 1 - Есть ли зависимость между количеством детей и возвратом кредита в срок?

Рассчитывать корреляцию Пирсона в данном случае (во всех четырёх вопросах) не эффективно, так как в данном случае одна из переменных категориальная (наличие долга).

**1) Изучим, какие значения принимает переменная children в данных.**

In [5]:
print(data_cleaned['children'].unique())

[1 0 3 2 4 5]


**2)  Изучим влияние числа детей на долю просрочек по кредиту.**

In [6]:
children_grouped = data_cleaned.groupby('children').agg({'debt': ['count', 'sum']})
children_debt_quality = children_grouped['debt']['sum'] / children_grouped['debt']['count']
children_debt_quality

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
dtype: float64

### Вывод:
- самые ненадежные заемщики с самой **высокой** долей просрочек 0.097 - это многодетные семьи с 4-мя детьми;
- самые надежные заемщики с самой **низкой** долей просрочек по кредиту 0.075 - это заемщики _без детей_.

Интересно, что для семей с пятью детьми, доля просрочек как будто равна нулю. Однако, судя по тому, что для многодетных детей с 4-мя детьми доля просрочек максимальная, было бы ошибочным сделать вывод о том, что многодетные семьи с 5-ю детьми являются более надежными заемщики, чем оные с четырьмя. Скорее всего, дело в устройстве датасета: данные по количеству детей есть, а данные по долгам отсутствуют.

### Вопрос 2 - Есть ли зависимость между семейным положением и возвратом кредита в срок?

**1) Изучим, какие значения принимает переменная family_status в данных.**

In [8]:
print(data_cleaned['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


_Понимая, какие значения принимает категориальная переменная family_status, группируем и посчитаем долю просрочек по кредиту._

In [9]:
family_grouped = data_cleaned.groupby('family_status').agg({'debt': ['count', 'sum']})
family_debt_quality = family_grouped['debt']['sum'] / family_grouped['debt']['count']
family_debt_quality

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
dtype: float64

### Вывод:

- одинокие заемщики без семьи (family_status = 'Не женат / не замужем') имеют наибольшую долю просрочек по кредиту 0.097. Это неожиданно, ведь выше был вывод о том, что семьи БЕЗ детей платят кредит исправно. Однако теперь можно расширить вывод о том, что мало не иметь детей, чтобы являться надежым заемщиком для банка, надо еще и не быть одиноким (быть женатым / замужем);
- наименьшая доля просрочек 0.66 по кредиту выявлена у овдовевших супругов (family_status = 'вдовец / вдова').

### Вопрос 3 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Уточним существующие категории заемщиков по уровню дохода:**
- 0–30000 — 'E'; 
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

***Долю просрочек в зависимости от категории дохода определим ниже:***

In [11]:
total_income_grouped = data_cleaned.groupby('total_income_category').agg({'debt':['sum','count']})
total_income_quality = total_income_grouped['debt']['sum']/total_income_grouped['debt']['count']
total_income_quality

total_income_category
A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
dtype: float64

**Выводы:**

- видим, что самая высокая доля просрочек по кредиту (0.09) у потенциальных заемщиков из категоии 'Е' с уровнем дохода 0-30000 руб;
- далее по росту надежности следуют потенциальные заемщики из категорий 'А' с доходом выше 1000000 руб (доля просрочек = 0.08) и 'С' с доходом 50001–200000 руб (доля просрочек = 0.084);
- наконец, самые надежные заемщики попадают в категорию 'D' с доходом 30001–50000 руб (доля просрочек = 0.06).

Получается, что исправнее всего кредит платят заемщики с доходом 30001–50000 руб. Учитывая, что Росстат на 4 кв. 2021 года заявил среднюю зарплату по России ~ 54000 руб, получается, что самые надежные заемщики, которые зарабатывают ниже среднего по России. Данный вывод сложно обобщить для всей России, рекомендовано исследование по каждому региону РФ.

### Вопрос 4 - Как разные цели кредита влияют на его возврат в срок?

In [12]:
purpose_grouped = data_cleaned.groupby('purpose_category').agg({'debt':['sum','count']})
purpose_debt = purpose_grouped['debt']['sum']/purpose_grouped['debt']['count']
print(purpose_debt.sort_values(ascending=False))

purpose_category
операции с автомобилем      0.093480
получение образования       0.092528
проведение свадьбы          0.079118
операции с недвижимостью    0.072551
dtype: float64


**Вывод:**

- самые ненадежные заемщики те, кто берут кредиты на образование (доля просрочки 0.092) и покупку автомобиля (0.93);
- далее по росту надежности следуют заемщики, которые берут кредиты на проведение свадьбы (доля просрочки = 0.079);
- самые надежные заемщики - те, кто берут кредиты на операции с недвижимостью (доля просрочки = 0.072).

### _Пояснения_

_**Возможные причины появления пропусков в исходных данных:** ручные ошибки ввода данных; необъективность заполнения анкеты на кредитование клиентами или сотрудниками банка; технический сбой._

_**Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных?** Медиана исключает искажение результатов: если появится эксперимент со аномальным значением, а у нас все равно будет использоваться медиана, то на валидность результатов это не отразится._

## 5. Общие выводы проекта

### 5.1  Идеальные заемщики (с наименьшей долей просрочки) для банка при рассмотрении заявки на кредиты - это:
- желающие взять ипотечный кредит (не автокредит, не кредит на образование и не автокредит);
- семейные люди без детей, желательно вдовец или вдова;
- с уровнем дохода на 10-15% ниже среднестатического по России 30001–50000 руб.

### 5.2  Самые ненадежные заемщики (с наибольшей долей просрочки) для банка при рассмотрении заявки на кредиты - это:
- желающие взять кредит на автомобиль или кредит на образование;
- одинокие люди (family_status = 'Не женат / не замужем');
- семьи с 4-мя детьми; важно отметить, что семьи с ребенком тоже имеют просрочки, идут на втором и третьем месте по доле просрочек после многодетных семей с 4-мя детьми;
- с уровнем дохода ниже 30000 руб (категория дохода 'Е').

_Your questions or comments regarding the project and data analysis are more than welcome to my email **ybezginova2021@gmail.com** or Telegram ***@ybezginova***_

_Best wishes,_
_Yulia_